In [1]:
from hjst_model.config import DatasetKVSConfig, LayerModelConfig, HierarchicalModelConfig
import numpy as np
import scipy
import nmslib
import pandas as pd
import os
import re

#カラム内の文字数。デフォルトは50だった
pd.set_option("display.max_colwidth", 800)
pd.set_option("display.width", 3)

#行数
pd.set_option("display.max_rows", 101)

def coloring(threshold, val):
    ret = []
    for i, v in enumerate(val):
        if not isinstance(v, str) and v < threshold:
            ret[0] = 'background-color: yellow' 
            ret[-1] = 'background-color: yellow'
            ret.append('background-color: yellow')
        else:
            ret.append('background-color: white')
    return ret

In [2]:
TESTSET = "aichi"
CONF_DIR = './configs'
HMODEL_NAME = 'dts16'

testset_conf = DatasetKVSConfig()
testset_conf.link(os.path.join(CONF_DIR, "dataset.conf"))
testset_conf.change_section(TESTSET)
testset = testset_conf.prepare_dataset(False)

# prepare layer config instance
layerconf = LayerModelConfig()
layerconf.link(os.path.join(CONF_DIR, "layer.conf"))

model_conf = HierarchicalModelConfig()
model_conf.link(os.path.join(CONF_DIR, HMODEL_NAME+'.conf'), create_if_missing=True)
hmodel = model_conf.model_generate()
#hmodel.topk_comptable(testset, '23/230006/0001/Law(1)', 2, 0.2, 'Sentence')

set layer: Doc2VecLayer 0.0
set layer: Doc2VecLayer 0.0
set layer: Doc2VecLayer 0.0


In [3]:
del testset
testset_conf.change_section("aichi")
testset = testset_conf.prepare_dataset(False)

In [3]:
test_sentence_layers = {l:{'layer': layerconf.load_model(l+'-Sentence'), 'threshold': th} for (l, th) in [
    ('swem_average_tokai_mecab', 0.2),
    ('swem_concat_tokai_mecab', 0.2),
    ('swem_max_tokai_mecab',0.01),
    ('swem_average_tokai_spm_8000',0.2),
    ('swem_concat_tokai_spm_8000',0.2),
    ('swem_average_tokai_spm_16000',0.2),
    ('swem_concat_tokai_spm_16000',0.2),
    ('tf_tokai_spm_8000',0.5),
    ('tfidf_tokai_spm_8000',0.5)
]}

In [49]:
l = 'd2v_tokai_mecab'
test_sentence_layers[l] = {'layer': layerconf.load_model(l+'-Sentence')}

In [ ]:

# get similar law tags and vectors(= simple method output)
testset.set_iterator_mode(level='Sentence', tag=True, sentence=False)
tags = list(testset)
for k in test_sentence_layers.keys():
    print(k)
    test_sentence_layers[k]['vectors'] = np.matrix([test_sentence_layers[k]['layer'][tag] for tag in tags])
    index = nmslib.init(method='hnsw', space='cosinesimil')
    index.addDataPointBatch(test_sentence_layers[k]['vectors'])
    index.createIndex({'post': 2}, print_progress=True)
    test_sentence_layers[k]['index'] = index
ltag_gen = (ltag for ltag in list(testset))

swem_average_tokai_mecab


In [ ]:

query = next(ltag_gen)
k = 1
threshold = 0.3

def get_caption(query):
    try:
        return testset.get_elem(query+"/{}Caption(1)".format(re.split("\(", os.path.split(query)[1])[0])).text
    except Exception as e:
        return ""
print("query:", str(testset.get_elem(law_tags[i])), get_caption(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
for k, d in test_sentence_layers.items():
    print(k)
    df = pd.DataFrame(
        [( str(testset.get_elem(law_tags[i])), get_caption(law_tags[i]), round(s, 3), testset.get_elem(law_tags[i]).text) for i, s in zip(*list( d['index'].knnQuery(d['layer'][query], 20) ))],
        columns = ["name", "caption", "similarity", "text"]
    )
    display(df)


In [6]:
lt = None
for ltag in testset:
    if "暴力団" in str(testset.get_elem(ltag)):
        lt = ltag
        print(lt, str(testset.get_elem(lt)))

23/230006/1797/Law(1) 愛知県公安委員会の権限に属する暴力団員による不当な行為の防止等に関する法律に規定する事務の一部を警察本部長等に委任する規則
23/230006/1813/Law(1) 愛知県暴力団排除条例
23/230006/1814/Law(1) 愛知県暴力団排除条例施行規則
23/231002/0999/Law(1) 名古屋市暴力団排除条例
23/232017/0086/Law(1) 豊橋市暴力団排除条例
23/232025/0692/Law(1) 岡崎市暴力団排除条例
23/232033/0700/Law(1) 一宮市暴力団等の排除に関する条例
23/232041/0562/Law(1) 瀬戸市暴力団排除条例
23/232050/0493/Law(1) 半田市暴力団排除条例
23/232068/0614/Law(1) 春日井市暴力団排除条例
23/232076/0498/Law(1) 豊川市暴力団排除条例
23/232084/0069/Law(1) 津島市暴力団排除条例
23/232092/0388/Law(1) 碧南市暴力団排除条例
23/232106/0325/Law(1) 刈谷市暴力団排除条例
23/232114/0797/Law(1) 豊田市暴力団排除条例
23/232122/0380/Law(1) 安城市暴力団排除条例
23/232131/0401/Law(1) 西尾市暴力団排除条例
23/232149/0555/Law(1) 蒲郡市暴力団排除条例
23/232157/0545/Law(1) 犬山市暴力団排除条例
23/232165/0610/Law(1) 常滑市暴力団排除条例
23/232173/0447/Law(1) 江南市暴力団排除条例
23/232190/0528/Law(1) 小牧市暴力団排除条例
23/232203/0089/Law(1) 稲沢市暴力団排除条例
23/232211/0595/Law(1) 新城市暴力団排除条例
23/232246/0460/Law(1) 知多市暴力団排除条例
23/232254/0273/Law(1) 知立市暴力団排除条例
23/232262/0345/Law(1) 尾張旭市暴力団排除条例
23/232271/0570/Law(1) 高浜市暴力団排除条例
23/232289/03

In [29]:
print(lt)

23/231002/0090/Law(1)


In [7]:
query = "23/230006/1813/Law(1)"
k = 100
root_threshold =  1000
threshold = 0.3
print("query:", testset.get_elem(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
tidx, distance = law_index.knnQuery(hmodel.layers[qlevel][query], k=k+1)
target_law_tags = [query]
for i, dist in zip(tidx, distance):
    if law_tags[i] == query:
        continue
    if root_threshold and dist > root_threshold:
        print("not enough distance", testset.get_elem(law_tags[i]), dist)
        k -= 1
        continue
    print("add target:", testset.get_elem(law_tags[i]), dist)
    target_law_tags.append(law_tags[i])

query: 愛知県暴力団排除条例
add target: 扶桑町高齢者作業所の設置及び管理に関する条例 0.617149
add target: 半田市農業振興基金条例 0.633147
add target: あま市美和総合福祉センターすみれの里条例施行規則 0.635461
add target: 小牧市保健センターの管理に関する規則 0.642202
add target: 一宮市教育委員会聴聞手続規則 0.642367
add target: 瀬戸市聴聞手続規則 0.643637
add target: 尾張旭市リサイクル広場の管理に関する規則 0.645386
add target: 小牧市土地区画整理事業施行地区内における建築行為等の許可に関する規則 0.645812
add target: 新城市公益的法人等への職員の派遣に関する条例 0.646441
add target: 知立市聴聞手続規則 0.646885
add target: 美浜町非常勤消防団員等に係る損害補償の支給等に関する規則 0.646928
add target: 扶桑町職員の公益的法人等への派遣に関する条例 0.647081
add target: 名古屋市研究開発地区建築条例 0.647721
add target: 江南市社会福祉基金の設置、管理及び処分に関する条例 0.648281
add target: 美浜町安全で住みよいまちづくり推進協議会規則 0.648831
add target: 春日井市の休日を定める条例 0.65075
add target: 東浦町行政手続条例 0.651166
add target: 幸田町非常勤消防団員等に係る損害補償の支給等に関する規則 0.653425
add target: 扶桑町聴聞手続規則 0.653742
add target: 岡崎市職員の職務に専念する義務の特例に関する条例 0.654169
add target: 蒲郡市消防団員等公務災害補償条例第9条の2第1項の規則で定める金額を定める規則 0.654255
add target: 高浜市聴聞手続規則 0.654268
add target: 岡崎市聴聞手続規則 0.654862
add target: 職員の服務の宣誓に関する条例 0.655326
add tar

In [48]:
target_law_tags = ['23/230006/1813/Law(1)', '23/231002/0999/Law(1)']
query = target_law_tags[0]

In [10]:
threshold = 0.3
d = test_sentence_layers['swem_average_tokai_mecab']
hmodel.layers['Sentence'] = d['layer']
#df = hmodel.comptable(testset, query, target_law_tags[1:], 10, 'Sentence')
#display(df.style.apply(lambda x: coloring(d['threshold'], x), axis=1))
for k, d in test_sentence_layers.items():
    if 'swem' not in k:
        continue
    print(k)
    hmodel.layers['Sentence'] = d['layer']
    #for qi, q in enumerate(target_law_tags):
    #    df = hmodel.comptable(testset, q, target_law_tags[:qi]+target_law_tags[qi+1:], 10, 'Sentence')
    #    display(df.style.apply(lambda x: coloring(threshold, x), axis=1))
    df = hmodel.comptable(testset, query, target_law_tags[1:], 10, 'Sentence')
    display(df.style.apply(lambda x: coloring(d['threshold'], x), axis=1))

swem_average_tokai_mecab


swem_concat_tokai_mecab


swem_max_tokai_mecab


swem_average_tokai_spm_8000


swem_concat_tokai_spm_8000


swem_average_tokai_spm_16000


swem_concat_tokai_spm_16000


In [5]:
# get similar law tags and vectors(= simple method output)
testset.set_iterator_mode(level='Law', tag=True, sentence=False)
law_tags = list(testset)
_, law_index = hmodel.create_index('Law', law_tags)
ltag_gen = (ltag for ltag in list(testset))

In [100]:
query = next(ltag_gen)
k = 1
threshold = 0.3
def get_caption(query):
    try:
        return testset.get_elem(query+"/{}Caption(1)".format(re.split("\(", os.path.split(query)[1])[0])).text
    except Exception as e:
        return ""
print("query:", str(testset.get_elem(law_tags[i])), get_caption(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
pd.DataFrame(
    [( str(testset.get_elem(law_tags[i])), get_caption(law_tags[i]), round(s, 3), testset.get_elem(law_tags[i]).text) for i, s in zip(*list( law_index.knnQuery(hmodel.layers[qlevel][query], 20) ))],
    columns = ["name", "caption", "similarity", "text"]
)

query: 豊明市公告式条例第1条第1項第1文 


,name,caption,similarity,text
0,愛知県公報規則第6条第1項第2号第1文,,0.000,愛知県議会議員
1,稲沢市職員の旅費の支給に関する規則第10条第1項第1号第1文,,0.003,愛知県内
2,愛西市職員等の旅費の支給に関する規則第11条第1項第1号第1文,,0.003,愛知県内
3,愛西市職員等の旅費の支給に関する規則第11条第2項第1号第1文,,0.003,愛知県内
4,愛知芸術文化センター条例第1条第2項第2号第1文,,0.003,愛知県芸術劇場
5,豊根村役場の位置を変更する条例第1項第2文,,0.004,愛知県北設楽郡豊根村下黒川字蕨平2
6,豊根村農村情報連絡施設の設置及び管理に関する条例第2条第1項第4号1号細分第2文,,0.004,愛知県北設楽郡豊根村下黒川字蕨平地内
7,豊根村農村情報連絡施設の設置及び管理に関する条例第2条第1項第4号1号細分第3文,,0.004,愛知県北設楽郡豊根村上黒川字金越地内
8,豊根村農村情報連絡施設の設置及び管理に関する条例第2条第1項第4号1号細分第4文,,0.004,愛知県北設楽郡豊根村坂宇場字宮嶋地内
9,豊根村農村情報連絡施設の設置及び管理に関する条例第2条第1項第4号1号細分第5文,,0.004,愛知県北設楽郡豊根村三沢字宮下地内


In [70]:
k = 1
root_threshold =  1000
threshold = 0.3
print("query:", testset.get_elem(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
tidx, distance = law_index.knnQuery(hmodel.layers[qlevel][query], k=k+1)
target_law_tags = [query]
for i, dist in zip(tidx, distance):
    if law_tags[i] == query:
        continue
    if root_threshold and dist > root_threshold:
        print("not enough distance", testset.get_elem(law_tags[i]), sim)
        k -= 1
        continue
    print("add target:", testset.get_elem(law_tags[i]), sim)
    target_law_tags.append(law_tags[i])
for qi, q in enumerate(target_law_tags):
    df = hmodel.comptable(testset, q, target_law_tags[:qi]+target_law_tags[qi+1:], 10, 'Sentence')
    display(df.style.apply(lambda x: coloring(threshold, x), axis=1))

query: 愛知県公告式条例第1条第1項第1文
add target: 豊明市公告式条例第1条第1項第1文 0.013063014


query: 議決事件指定条例
特別職の指定に関する条例 5.9604645e-08
議決事件指定条例 1.1920929e-07
沿道区域指定の基準に関する条例 1.1920929e-07
都市的施設その他の都市としての要件に関する条例 0.009170651
豊橋市新型インフルエンザ等対策本部条例 0.23891258
西尾市新型インフルエンザ等対策本部条例 0.23891258
職員の退職手当に関する条例の一部を改正する条例附則第二項の規定により退職手当の支給を受ける職員の範囲を定める規則 0.25446343
公立学校職員の退職手当に関する条例の一部を改正する条例附則第二項の規定により退職手当の支給を受ける職員の範囲を定める規則 0.26839417
南知多町における高齢者、障害者等の移動等の円滑化のために必要な特定公園施設の設置の基準に関する条例 0.31211913
武豊町移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.31211913
尾張旭市指定地域密着型サービス事業者及び指定地域密着型介護予防サービス事業者の指定に関する基準を定める条例 0.3121192
尾張旭市道路構造の技術的基準を定める条例 0.3121192
尾張旭市市道に設ける案内標識等の寸法を定める条例 0.3121192
尾張旭市準用河川管理施設等の構造の技術的基準を定める条例 0.3121192
尾張旭市移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.3121192
尾張旭市水道事業の布設工事監督者及び水道技術管理者に関する基準を定める条例 0.3121192
東郷町不育症治療費の助成に関する条例 0.3121192
東郷町移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.3121192
南知多町水道事業の布設工事監督者及び水道技術管理者に関する条例 0.3121192
西尾市教育委員会公告式規則 0.32603073


In [5]:
with open("./../reikiset/21/210005/0067.xml") as f:
    print(f.read())

﻿<?xml version="1.0" encoding="UTF-8"?>
<Laws DictVer="">
<Law OriginalPromulgateDate="昭和二十四年三月二十八日条例第十四号" LawType="" Lang="ja" Year="" Era="" Num="">
	<LawNum>昭和二十四年三月二十八日条例第十四号</LawNum>
	<LawBody>
		<LawTitle>岐阜県公衆浴場法施行条例</LawTitle>
		<EnactStatement>昭和二十三年七月法律第百三十九号公衆浴場法により、岐阜県公衆浴場法施行条例を、次のように定める。</EnactStatement>
		<MainProvision>
			<Article Num="1">
				<ArticleTitle>第一条</ArticleTitle>
				<Paragraph Num="1">
					<ParagraphNum />
					<ParagraphSentence>
						<Sentence>公衆浴場法第二条第三項の規定による公衆浴場の設置の場所の配置の基準並びに同法第三条第二項の規定による公衆浴場の換気、採光、照明保温及び清潔その他入浴者の衛生及び風紀について必要な措置の基準は、この条例の定めるところによる。</Sentence>
					</ParagraphSentence>
				</Paragraph>
			</Article>
			<Article Num="2">
				<ArticleTitle>第二条</ArticleTitle>
				<Paragraph Num="1">
					<ParagraphNum />
					<ParagraphSentence>
						<Sentence Num="1" Function="Main">新たに設置しようとする公衆浴場は、既設の公衆浴場との間に、次に掲げる距離を保たなければならない。</Sentence>
						<Sentence Num="2" Function="Proviso">ただし、土地の状況、予想利用者数、人口密度等を考慮して公衆浴場の設置が公衆衛生上必要であると知事が認めたときは、この限りでない。</

In [101]:
import sys 
import nmslib 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [ ]:
def read_data(layer, sample_size):
    
        

In [105]:
# Read data points
testset.set_iterator_mode(level='Sentence', tag=True, sentence=False)
all_data_matrix =  np.matrix([hmodel.layers["Sentence"][tag] for tag in testset])

In [125]:
sample_data_matrix = all_data_matrix#[:50000]
sample_data_matrix.shape

(610982, 500)

In [126]:
# Create a held-out query data set
(data_matrix, query_matrix) = train_test_split(sample_data_matrix, test_size = 0.01)

In [127]:
query_matrix.shape

(6110, 500)

In [128]:
# Set index parameters
# These are the most important onese
M = 30
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [129]:
# Number of neighbors 
K=100

In [130]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space='cosinesimil') 
index.addDataPointBatch(data_matrix) 

604872

In [131]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 30, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}
Indexing time = 185.833216


In [132]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params) 

Setting query-time parameters {'efSearch': 100}


In [133]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=0.544561 (sec), per query=0.000089 (sec), per query adjusted for thread number=0.000357 (sec)


In [134]:
# Computing gold-standard data 
print('Computing gold-standard data')

start = time.time()
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(data_matrix)
end = time.time()

print('Brute-force preparation time %f' % (end - start))

start = time.time() 
gs = sindx.kneighbors(query_matrix)
end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
Brute-force preparation time 0.157155
brute-force kNN time total=131.925448 (sec), per query=0.021592 (sec)


In [135]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.894360
